<a href="https://colab.research.google.com/github/Azhimanich/Data-Crawling/blob/main/Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.parse import urldefrag
import requests
import time
from bs4 import BeautifulSoup
from collections import OrderedDict

In [ ]:
class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls

    def download_url(self, url):
        s = requests.Session()
        headers = OrderedDict({
            'Accept-Encoding': 'gzip, deflate, br',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'
        })
        s.headers = headers
        return s.get(url, headers=headers).text

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')

        # MODIFIKASI SESUAI KEBUTUHAN
        for link in soup.find_all('a', {'class': 'article__link'}):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        defrag_parts = urldefrag(url)
        defrag_url = defrag_parts[0]
        if defrag_url not in self.visited_urls and defrag_url not in self.urls_to_visit:
            self.urls_to_visit.append(defrag_url)

    def crawl(self, url):
        html = self.download_url(url)

        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)

    def run(self):
        parsed_uri = urlparse(self.urls_to_visit[0])
        domain_uri = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)

        # MODIFIKASI SESUAI KEBUTUHAN
        max_num_of_crawls = 3
        num_of_crawls = 0

        while self.urls_to_visit and num_of_crawls < max_num_of_crawls:
            url = self.urls_to_visit.pop(0)
            parsed_url = urlparse(url)
            domain_url = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_url)

            if domain_uri == domain_url:
                time.sleep(1)
                try:
                    self.crawl(url)
                except Exception:
                    print(f'Failed to crawl: {url}')
                finally:
                    self.visited_urls.append(url)
            else:
                print(url)

                html = self.download_url(url)
                soup = BeautifulSoup(html, 'html.parser')
                content = soup.find("div", {"class": "read__content"})
                # Returns all the text in a document or beneath a tag, as a single Unicode string
                print(content.get_text())

            num_of_crawls = num_of_crawls + 1

In [ ]:
Crawler(urls=['https://kompas.com']).run()

https://tekno.kompas.com/read/2023/10/05/13400087/suasana-tiktok-live-setelah-keranjang-kuning-dihapus-masih-ramai-penjual-dan


KOMPAS.com - Layanan TikTok Shop yang ada di aplikasi TikTok di Indonesia resmi dihentikan pada Rabu (4/10/2023) pukul 17.00 WIB.
Artinya, pengguna TikTok saat ini tak bisa lagi menemukan beragam menu yang berkaitan dengan TikTok Shop, yaitu fitur "keranjang kuning" yang biasa dipakai untuk membeli barang, atau menu "Shop" itu sendiri. 
Meski TikTok Shop sudah ditutup, para pedagang tampaknya masih tetap berbondong-bondong berjualan dan menawarkan produknya masing-masing secara langsung melalui fitur siaran langsung alias TikTok Live.
Baca juga: Masih Ada Penjual yang Live di TikTok, tapi Tidak Bisa Transaksi
Pantauan KompasTekno pada Kamis (5/10/2023) pagi, ketika kami mengeklik ikon "Live" di aplikasi TikTok, kami langsung disuguhkan dengan konten TikTok Live yang sedang berjualan, meski sudah tak ada lagi fitur keranjang kuning.
Kemudian, kami mencoba meng